## Notebook Overview

This notebook builds upon the feature-engineered dataset from the previous notebook (`04_feature_engineering.ipynb`) and focuses on **Model Training and Evaluation**. Our primary goal is to develop a credit risk prediction model that excels at identifying potential loan defaulters, thereby minimizing financial losses for retail banks while also considering their desired balance between risk aversion and loan approval rates. This translates to maximizing the recall of the positive class (loan defaulters) while maintaining acceptable precision and overall model performance.

### 0.5.1 Objectives

The main objectives of this notebook are:

1. **Model Selection:** Choose algorithms suitable for imbalanced classification problems.
2. **Model Training:** Train models with a focus on identifying potential defaulters.
3. **Hyperparameter Tuning:** Optimize models to increase recall for the positive class.
4. **Model Evaluation:** Assess models primarily on recall, while considering precision, F2-score, AUC-PR, and overall performance.
5. **Model Comparison:** Compare different models based on their ability to identify true positives and balance the precision-recall trade-off.
6. **Threshold Adjustment:** Explore the impact of classification thresholds on recall and precision, collaborating with retail banks to determine the optimal threshold.

### 0.5.2 Importance of Focusing on Recall

Prioritizing recall for defaulter prediction is crucial for minimizing financial losses, which is the primary business objective in credit risk assessment. The cost of missing a potential defaulter (false negative) is typically much higher than the cost of incorrectly classifying a non-defaulter as high-risk (false positive). While we prioritize recall, we will also carefully consider the precision-recall trade-off and aim for a model that maximizes recall without severely impacting precision. Techniques like threshold adjustment and cost-sensitive learning will be used to balance these metrics effectively. Furthermore, demonstrating a thorough approach to risk identification aligns with regulatory expectations in the financial sector, supporting the banks' compliance needs. This approach also allows for more conservative lending practices, which can be adjusted based on the bank's specific risk tolerance.

### 0.5.3 Our Approach

In this notebook, we will focus on the following modeling tasks:

1. **Data Preparation:** Address class imbalance using techniques like SMOTE or class weighting.
2. **Baseline Model:** A logistic regression model with class weights inversely proportional to class frequencies will serve as our baseline. This will provide a benchmark for evaluating more complex models.
3. **Advanced Models:** Train and evaluate models known for handling imbalanced data:
   - Decision Trees with adjusted class weights
   - Random Forest with balanced class weights
   - Gradient Boosting (XGBoost, LightGBM) with `scale_pos_weight` adjustment
4. **Hyperparameter Tuning:** We will employ techniques like GridSearchCV or RandomizedSearchCV, optimizing for the F2-score (which gives more weight to recall) or a custom cost-sensitive scoring function.
5. **Model Evaluation:** Prioritize recall in our metrics, while also considering precision, F2-score, AUC-PR, and AUC-ROC.
6. **Threshold Adjustment:** We will experiment with different classification thresholds and work closely with retail banks to determine the optimal threshold that balances their desired level of risk aversion with acceptable loan approval rates.
7. **Ensemble Methods:** Explore ensemble techniques that can improve recall without severely impacting precision.
8. **Cost-Sensitive Learning:** Incorporate misclassification costs to reflect the higher cost of false negatives, aligning the model's objective with the business goal of minimizing financial losses.

By the end of this notebook, we aim to have a model (or ensemble of models) that excels at identifying potential loan defaulters, providing the bank with a powerful tool for risk assessment and mitigation.


In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
)
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    precision_recall_curve,
    auc,
    confusion_matrix,
    f1_score,
    recall_score,
    precision_score,
    make_scorer,
)
from retail_bank_risk.model_training_utils import downscale_dtypes
from retail_bank_risk.advanced_visualizations_utils import (
    plot_confusion_matrix,
    plot_model_performance,
    shap_summary_plot,
    shap_force_plot,
    plot_roc_curve,
    plot_precision_recall_curve,
    plot_combined_confusion_matrices,
    plot_learning_curve,
)

from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

import matplotlib.pyplot as plt
import seaborn as sns
import shap

After loading the necessary libraries, we load the training and test datasets and verify their dimensions and initial rows to ensure data integrity.

In [2]:
train_df = pd.read_parquet(
    "../data/processed/application_train_engineered.parquet"
)
test_df = pd.read_parquet(
    "../data/processed/application_test_engineered.parquet"
)

In [3]:
print(f"Training Data Shape: {train_df.shape}")
print(f"Test Data Shape: {test_df.shape}")

train_df.head()

Training Data Shape: (307511, 78)
Test Data Shape: (48744, 77)


,reg_city_not_work_city_0,reg_city_not_work_city_1,region_rating_client_w_city,region_rating_client,name_contract_type_cash loans,name_contract_type_revolving loans,code_gender_m,code_gender_f,flag_own_car_n,flag_own_car_y,...,is_anomaly_true,age_group,income_group,credit_amount_group,debt_to_income_ratio,credit_to_goods_ratio,annuity_to_income_ratio,ext_source_mean,credit_exceeds_goods,target
0,1,0,1.0,1.0,1,0,1,0,1,0,...,0,1.0,3.0,1.0,2.007889,1.158397,0.121978,0.201162,1,1
1,1,0,2.0,2.0,1,0,0,1,1,0,...,0,3.0,4.0,4.0,4.790750,1.145199,0.132217,0.587418,1,0
2,1,0,1.0,1.0,0,1,1,0,0,1,...,0,3.0,0.0,0.0,2.000000,1.000000,0.100000,0.642739,0,0
3,1,0,1.0,1.0,1,0,0,1,1,0,...,0,3.0,1.0,1.0,2.316167,1.052803,0.219900,0.579984,1,0
4,0,1,1.0,1.0,1,0,1,0,1,0,...,0,3.0,1.0,2.0,4.222222,1.000000,0.179963,0.381898,0,0


The data appears correctly processed: encoded variables are saved, new features are created, and the dataset shapes match expectations.

Next, we optimize data types to minimize memory usage without information loss.

In [4]:
train_df, test_df = downscale_dtypes(train_df, test_df, target_column='target')

train_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 78 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   reg_city_not_work_city_0                 307511 non-null  uint8  
 1   reg_city_not_work_city_1                 307511 non-null  uint8  
 2   region_rating_client_w_city              307511 non-null  float32
 3   region_rating_client                     307511 non-null  float32
 4   name_contract_type_cash loans            307511 non-null  uint8  
 5   name_contract_type_revolving loans       307511 non-null  uint8  
 6   code_gender_m                            307511 non-null  uint8  
 7   code_gender_f                            307511 non-null  uint8  
 8   flag_own_car_n                           307511 non-null  uint8  
 9   flag_own_car_y                           307511 non-null  uint8  
 10  flag_own_realty_y               

Next, we will be splitting the dataset into predictors and the target variable for modeling.

In [5]:
X = train_df.drop(["target", "sk_id_curr"], axis=1)
y = train_df["target"]

print(f"Number of Features: {X.shape[1]}")
print("Feature Names:", X.columns.tolist())

Number of Features: 76
Feature Names: ['reg_city_not_work_city_0', 'reg_city_not_work_city_1', 'region_rating_client_w_city', 'region_rating_client', 'name_contract_type_cash loans', 'name_contract_type_revolving loans', 'code_gender_m', 'code_gender_f', 'flag_own_car_n', 'flag_own_car_y', 'flag_own_realty_y', 'flag_own_realty_n', 'name_type_suite_unaccompanied', 'name_type_suite_family', 'name_type_suite_spouse, partner', 'name_type_suite_children', 'name_type_suite_other_a', 'name_type_suite_mode', 'name_type_suite_other_b', 'name_type_suite_group of people', 'name_income_type_working', 'name_income_type_state servant', 'name_income_type_commercial associate', 'name_income_type_pensioner', 'name_income_type_unemployed', 'name_income_type_student', 'name_income_type_businessman', 'name_income_type_maternity leave', 'name_education_type', 'name_family_status_single / not married', 'name_family_status_married', 'name_family_status_civil marriage', 'name_family_status_widow', 'name_famil

In [6]:
X_test = test_df.drop("sk_id_curr", axis=1)
sk_id_curr = test_df["sk_id_curr"]

print(f"Number of Features in test set: {X_test.shape[1]}")
print("Feature Names in test set:", X_test.columns.tolist())

Number of Features in test set: 76
Feature Names in test set: ['reg_city_not_work_city_0', 'reg_city_not_work_city_1', 'region_rating_client_w_city', 'region_rating_client', 'name_contract_type_cash loans', 'name_contract_type_revolving loans', 'code_gender_m', 'code_gender_f', 'flag_own_car_n', 'flag_own_car_y', 'flag_own_realty_y', 'flag_own_realty_n', 'name_type_suite_unaccompanied', 'name_type_suite_family', 'name_type_suite_spouse, partner', 'name_type_suite_children', 'name_type_suite_other_a', 'name_type_suite_mode', 'name_type_suite_other_b', 'name_type_suite_group of people', 'name_income_type_working', 'name_income_type_state servant', 'name_income_type_commercial associate', 'name_income_type_pensioner', 'name_income_type_unemployed', 'name_income_type_student', 'name_income_type_businessman', 'name_income_type_maternity leave', 'name_education_type', 'name_family_status_single / not married', 'name_family_status_married', 'name_family_status_civil marriage', 'name_family_st

Furthermore, we divide the data to evaluate model performance on unseen data while preserving class distribution.

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training Set Shape: {X_train.shape}")
print(f"Validation Set Shape: {X_val.shape}")

Training Set Shape: (246008, 76)
Validation Set Shape: (61503, 76)


Before optimizing and tuning complex models, it's essential to establish baseline models.

These models will provide a benchmark against which you can measure the performance of more sophisticated models and tuning strategies.

We will start by defining the baseline pipelines.

In [8]:
import re

def sanitize_feature_names(X):
    return X.rename(columns=lambda x: re.sub(r'[^\w]+', '_', x))

In [9]:
from sklearn.preprocessing import FunctionTransformer

sanitize_transformer = FunctionTransformer(sanitize_feature_names)

In [10]:
pipeline_dummy = Pipeline([
    ('sanitize', sanitize_transformer),
    ('classifier', DummyClassifier(strategy='most_frequent'))
])

pipeline_lr = Pipeline([
    ('sanitize', sanitize_transformer),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

pipeline_dt = Pipeline([
    ('sanitize', sanitize_transformer),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

pipeline_rf = Pipeline([
    ('sanitize', sanitize_transformer),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
])

pipeline_xgb = Pipeline([
    ('sanitize', sanitize_transformer),
    ('classifier', xgb.XGBClassifier(
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_jobs=-1
    ))
])

pipeline_lgb = Pipeline([
    ('sanitize', sanitize_transformer),
    ('classifier', lgb.LGBMClassifier(
        objective='binary',
        random_state=42,
        n_jobs=-1
    ))
])

Next, we will organize models into a list for streamlined training and evaluation.

In [11]:
baseline_models = [
    ('Dummy Classifier', pipeline_dummy),
    ('Logistic Regression', pipeline_lr),
    ('Decision Tree', pipeline_dt),
    ('Random Forest', pipeline_rf),
    ('XGBoost', pipeline_xgb),
    ('LightGBM', pipeline_lgb)
]

Next we will run the function which will automate the training and evaluation process, providing key metrics and visualizations for each model.

In [12]:
from joblib import Parallel, delayed
import os
import numpy as np
from sklearn.metrics import (
    recall_score, precision_score, f1_score, roc_auc_score,
    precision_recall_curve, auc, classification_report, confusion_matrix
)

def train_evaluate_model(name, pipeline, X_train, y_train, X_val, y_val):
    print(f"Training {name}...")
    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_val)
    y_pred_proba = pipeline.predict_proba(X_val)[:, 1]

    recall = recall_score(y_val, y_pred, zero_division=0)
    precision = precision_score(y_val, y_pred, zero_division=0)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    precision_vals, recall_vals, _ = precision_recall_curve(y_val, y_pred_proba)
    pr_auc = auc(recall_vals, precision_vals)

    print(f"\nClassification Report for {name}:")
    print(classification_report(y_val, y_pred))

    cm = confusion_matrix(y_val, y_pred)
    print(f"Confusion Matrix for {name}:\n{cm}\n")

    print(f"{name} Metrics:")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC-PR: {pr_auc:.4f}")
    print(f"AUC-ROC: {roc_auc:.4f}\n")

    os.makedirs("../images/model_performance/", exist_ok=True)
    base_path = f"../images/model_performance/{name.lower().replace(' ', '_')}"

    # ROC Curve
    save_path = f"{base_path}_roc_curve.png"
    plot_roc_curve(y_val, y_pred_proba, save_path=save_path)
    print(f"ROC curve saved to {save_path}")

    # Precision-Recall Curve
    save_path = f"{base_path}_pr_curve.png"
    plot_precision_recall_curve(y_val, y_pred_proba, save_path=save_path)
    print(f"Precision-Recall curve saved to {save_path}")

    # Learning Curve
    save_path = f"{base_path}_learning_curve.png"
    plot_learning_curve(
        pipeline,
        X_train,
        y_train,
        cv=5,
        n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 5),
        save_path=save_path
    )
    print(f"Learning curve saved to {save_path}")

    print("="*60 + "\n")

    return {
        'model': name,
        'pipeline': pipeline,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'recall': recall,
        'precision': precision,
        'f1_score': f1,
        'auc_pr': pr_auc,
        'auc_roc': roc_auc
    }

# Define number of parallel jobs based on your CPU cores
parallel_jobs = 6   # Adjust as needed

# Train and evaluate all baseline models in parallel
baseline_results = Parallel(n_jobs=parallel_jobs)(
    delayed(train_evaluate_model)(name, pipeline, X_train, y_train, X_val, y_val)
    for name, pipeline in baseline_models
)

# After all models have been trained and evaluated
def plot_all_model_performance(results):
    metrics = ['recall', 'precision', 'f1_score', 'auc_pr', 'auc_roc']
    plot_model_performance(
        {model['model']: {metric: model[metric] for metric in metrics} for model in results},
        metrics,
        save_path="../images/model_performance/model_performance_comparison.png"
    )
    print("Model performance comparison plot saved to ../images/model_performance/model_performance_comparison.png")

def plot_all_confusion_matrices(results, y_val):
    y_pred_dict = {model['model']: model['y_pred'] for model in results}
    plot_combined_confusion_matrices(
        {model['model']: {metric: model[metric] for metric in ['recall', 'precision', 'f1_score', 'auc_pr', 'auc_roc']} for model in results},
        y_val,
        y_pred_dict,
        labels=["Non-Defaulter", "Defaulter"],
        save_path="../images/model_performance/combined_confusion_matrices.png"
    )
    print("Combined confusion matrices plot saved to ../images/model_performance/combined_confusion_matrices.png")

# Generate combined plots
plot_all_model_performance(baseline_results)
plot_all_confusion_matrices(baseline_results, y_val)

Training Dummy Classifier...
Training Logistic Regression...

Classification Report for Dummy Classifier:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.00      0.00      0.00      4965

    accuracy                           0.92     61503
   macro avg       0.46      0.50      0.48     61503
weighted avg       0.85      0.92      0.88     61503

Confusion Matrix for Dummy Classifier:
[[56538     0]
 [ 4965     0]]

Dummy Classifier Metrics:
Recall: 0.0000
Precision: 0.0000
F1 Score: 0.0000
AUC-PR: 0.5404
AUC-ROC: 0.5000

Training Decision Tree...


/Users/vytautasbunevicius/retail-bank-risk-evaluation/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/vytautasbunevicius/retail-bank-risk-evaluation/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/vytautasbunevicius/retail-bank-risk-evaluation/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

Training Random Forest...
Training XGBoost...
Training LightGBM...
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045758 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3532
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.52      0.01      0.02      4965

    accuracy                           0.92     61503
   macro avg       0.72      0.50      0.49     61503
weighted avg       0.89      0.92      0.88     61503

/Users/vytautasbunevicius/retail-bank-risk-evaluation/venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:58:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion matrix saved to ../images/model_performance/logistic_regression_confusion_matrix.png
ROC curve saved to ../images/model_performance/logistic_regression_roc_curve.png
Precision-Recall curve saved to ../images/model_performance/logistic_regression_pr_curve.png
Learning curve saved to ../images/model_performance/dummy_classifier_learning_curve.png


Classification Report for Decision Tree:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56538
           1       1.00      1.00      1.00      4965

    accuracy                           1.00     61503
   macro avg       1.00      1.00      1.00     61503
weighted avg       1.00      1.00      1.00     61503

Confusion Matrix for Decision Tree:
[[56526    12]
 [    8  4957]]

Decision Tree Metrics:
Recall: 0.9984
Precision: 0.9976
F1 Score: 0.9980
AUC-PR: 0.9981
AUC-ROC: 0.9991

Confusion matrix saved to ../images/model_performance/decision_tree_confusion_matrix.png
ROC curve s

...

[LightGBM] [Info] Number of positive: 12271, number of negative: 140253
[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
Confusion matrix saved to ../images/model_performance/random_forest_confusion_matrix.png
[LightGBM] [Info] Number of positive: 12271, number of negative: 140253
[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059762 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3537
[LightGBM] [Info] Number of data points in the train set: 152524, number of used features: 71
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058584 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3533
[LightGBM] [Info] Number of data points in the train set: 152524, number 

[19:59:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" }

Learning curve saved to ../images/model_performance/decision_tree_learning_curve.png



[19:59:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[19:59:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



Learning curve saved to ../images/model_performance/xgboost_learning_curve.png

[LightGBM] [Info] Number of positive: 1577, number of negative: 18103
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016462 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3474
[LightGBM] [Info] Number of data points in the train set: 19680, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080132 -> initscore=-2.440553
[LightGBM] [Info] Start training from score -2.440553
[LightGBM] [Info] Number of positive: 5095, number of negative: 58866
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.086189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3511
[LightGBM] [Info] Number of data 

In [13]:
# # Define number of parallel jobs based on your CPU cores
# parallel_jobs = 6   # Adjust as needed

# # Train and evaluate all baseline models in parallel
# baseline_results = Parallel(n_jobs=parallel_jobs)(
#     delayed(train_evaluate_model)(name, pipeline, X_train, y_train, X_val, y_val)
#     for name, pipeline in baseline_models
# )

Define pipelines for different models.

This way, we ensure the same preprocessing steps are applied within each model's evaluation during cross-validation, preventing data leakage and making results more reliable.


Define parameter grids for each model
